#  Few-shot 프롬프팅

---

# 환경 설정 및 준비

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import json
from langfuse import get_client
from langfuse.langchain import CallbackHandler
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Langfuse 설정
langfuse = get_client()
langfuse_handler = CallbackHandler()

# LLM 설정
llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3,
)

/Users/kwonsoonhyun/Sesac/004_llm_agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/kwonsoonhyun/Sesac/004_llm_agent/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_

# **Zero-shot** 프롬프팅

- **Zero-shot 프롬프팅**은 예시 없이 AI가 즉시 작업을 수행하는 기법입니다

- 명확한 **지시사항**만으로 원하는 결과를 얻을 수 있어 **사용이 간단**합니다

- 단순하고 직관적인 작업에 적합한 프롬프팅 방식이지만, 작업의 **복잡도에 따라 선택적 사용**이 필요합니다

`(1) without context`

In [5]:
# Zero-shot 프롬프트 생성 #예시없이 사용한다.
langfuse.create_prompt(
    name="competitor-analysis-zero-shot",
    type="text",
    prompt="다음 시장에서 삼성전자의 경쟁업체를 설명해주세요: {{topic}}", #토픿변수 랭퓨즈는 이중괄호
    labels=["production"],
    tags=["competitor", "analysis", "zero-shot"] #태그 필터링
)

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Zero-shot 프롬프트 템플릿 사용
zero_shot_template = langfuse.get_prompt("competitor-analysis-zero-shot")

# Langfuse 프롬프트를 LangChain과 통합
zero_shot_prompt = PromptTemplate.from_template(
    zero_shot_template.get_langchain_prompt()
)
zero_shot_prompt.metadata={"langfuse_prompt": zero_shot_template}  # Langfuse 자동 링크를 위한 메타데이터

# 체인 생성
chain = zero_shot_prompt | llm | StrOutputParser()

zero_shot_result = chain.invoke(
    {"topic": "인공지능 반도체"}, #토픽으로 인공지능 전달
    config={"callbacks": [langfuse_handler]} #초기화했던거 전달해주면 체인의 실행내용을 트래이싱 추적할 수 있다. 체인이 실행되고 인공지능 반도체 내용 아는거 llm이말함
)

print(zero_shot_result)

인공지능 반도체 시장에서 삼성전자의 주요 경쟁업체는 다음과 같습니다:

1. **엔비디아 (NVIDIA)**  
   - 인공지능 및 딥러닝 분야에서 가장 선도적인 GPU 제조업체입니다. 특히 데이터센터용 AI 가속기와 자율주행, 로보틱스 등 다양한 AI 응용 분야에 강점을 가지고 있습니다.

2. **인텔 (Intel)**  
   - CPU뿐만 아니라 AI 가속기, FPGA, 그리고 AI 전용 칩 개발에 적극 투자하고 있습니다. 최근에는 AI 반도체 스타트업 인수와 자체 AI 칩 개발을 통해 시장 경쟁력을 강화하고 있습니다.

3. **구글 (Google)**  
   - TPU(Tensor Processing Unit)라는 AI 전용 반도체를 자체 개발하여 클라우드 AI 서비스에 활용하고 있습니다. 구글 클라우드 플랫폼을 통한 AI 반도체 서비스도 제공 중입니다.

4. **AMD**  
   - 고성능 GPU를 기반으로 AI 및 머신러닝 가속기 시장에 진출하고 있으며, 엔비디아와 경쟁 구도를 형성하고 있습니다.

5. **화웨이 (Huawei)**  
   - 자체 AI 칩셋인 Ascend 시리즈를 개발하여 데이터센터 및 엣지 컴퓨팅용 AI 반도체 시장을 공략하고 있습니다.

6. **미디어텍 (MediaTek)**  
   - 모바일 및 엣지 AI 반도체 시장에서 경쟁력을 갖추고 있으며, AI 기능이 강화된 칩셋을 출시하고 있습니다.

이 외에도 여러 스타트업과 반도체 기업들이 AI 반도체 시장에 진출하고 있어 경쟁이 점차 치열해지고 있습니다. 삼성전자는 메모리 반도체뿐만 아니라 AI 연산에 최적화된 비메모리 반도체 개발에도 집중하며 이 시장에서 경쟁력을 강화하고 있습니다.


`(2) with context`

In [7]:
# Zero-shot with context 프롬프트 생성 
langfuse.create_prompt(
    name="competitor-analysis-zero-shot-context",
    type="text", ##컨텍스트를 제공하는것 뿐이지 원샷은아니고 제로샷이다.
    prompt="""{{topic}} 시장에서 삼성전자의 경쟁업체를 설명해주세요. 
반드시 다음 제시된 뉴스에 근거해서 답변하세요:

[뉴스]
{{context}} 

[답변]
""",
    labels=["production"],
    tags=["competitor", "analysis", "zero-shot", "context"]
)

In [8]:
# Zero-shot with context 프롬프트 템플릿 사용
zero_shot_template = langfuse.get_prompt("competitor-analysis-zero-shot-context")

# Langfuse 프롬프트를 LangChain과 통합
zero_shot_prompt = PromptTemplate.from_template(
    zero_shot_template.get_langchain_prompt()
)
zero_shot_prompt.metadata={"langfuse_prompt": zero_shot_template}  # Langfuse 자동 링크를 위한 메타데이터


# 체인 생성
chain = zero_shot_prompt | llm | StrOutputParser()

# Zero-shot 실행
context = """삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 
이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 
세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.
"""

topic = "인공지능 반도체"
zero_shot_result = chain.invoke(
    {"context": context, "topic": topic},
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백
    )

print(zero_shot_result)

[답변]  
인공지능 반도체 시장에서 삼성전자의 주요 경쟁업체는 현재 지배적인 위치를 차지하고 있는 엔비디아입니다. 삼성전자는 내년 초 자체 개발한 AI 가속기를 출시하여 엔비디아의 독점적 지위에 도전하고, 세계 최고의 반도체 제조업체로서의 위상을 강화하려는 전략을 추진하고 있습니다. 따라서 엔비디아가 삼성전자의 가장 큰 경쟁자로 볼 수 있습니다.


# **One-shot** 프롬프팅

- **One-shot 프롬프팅**은 하나의 예시를 통해 AI가 작업 패턴을 학습하는 기법입니다

- **Zero-shot** 방식보다 더 나은 성능을 제공하며, **형식화된 작업**에 특히 효과적입니다

- 단일 예시로 **품질 향상**이 가능하나, 해당 예시에 **과의존**할 수 있는 한계가 있습니다

In [9]:
### One-shot 프롬프트 템플릿 생성
# 1. Zero-shot 프롬프트 템플릿에 예시(example)를 포함하도록 수정
# 2. input_variables에 example_topic과 example_response 추가
# 3. config에 예시들을 저장

# 예시
example_topic = "스마트폰"
example_response = "애플: 프리미엄 시장에서 주요 경쟁사로, iPhone 시리즈로 경쟁\n샤오미: 중저가 시장에서 강세를 보이며 글로벌 시장 점유율 확대\n구글: Pixel 시리즈로 프리미엄 시장 진출, AI 기능 강조"
print(example_response) #프롬프트 자체로 랭퓨즈에 등록을해준다. 

langfuse.create_prompt(
    name="competitor-analysis-one-shot",
    type="text",
    prompt="""다음은 특정 시장에서 삼성전자의 경쟁업체를 설명하는 예시이다:

시장: {{example_topic}}
경쟁업체: {{example_response}}

이제 다음 시장에서 삼성전자의 경쟁업체를 설명해주세요:
시장: {{topic}}""",
    labels=["production"],
    tags=["competitor", "analysis", "one-shot"],
    config={
        "example_topic": example_topic,
        "example_response": example_response #프롬프트의 컨피그 값을 가져와서 예시로보여주고 그럼 모델 대답도 예시처럼 따라간다 
    }
)

애플: 프리미엄 시장에서 주요 경쟁사로, iPhone 시리즈로 경쟁
샤오미: 중저가 시장에서 강세를 보이며 글로벌 시장 점유율 확대
구글: Pixel 시리즈로 프리미엄 시장 진출, AI 기능 강조


In [10]:
# 프롬프트 템플릿 사용
one_shot_template = langfuse.get_prompt("competitor-analysis-one-shot")

# Langfuse 프롬프트를 LangChain과 통합
one_shot_prompt = PromptTemplate.from_template(
    one_shot_template.get_langchain_prompt()
)
one_shot_prompt.metadata={"langfuse_prompt": one_shot_template}  # Langfuse 자동 링크를 위한 메타데이터

# one_shot_prompt 적용한 체인 생성
chain = one_shot_prompt | llm | StrOutputParser()

# One-shot 실행
topic = "인공지능 반도체"
one_shot_result = chain.invoke(
    input={
        "example_topic": one_shot_template.config.get("example_topic"),
        "example_response": one_shot_template.config.get("example_response"),
        "topic": topic
    },
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백
)

print(f"one_shot_result:")
print(one_shot_result)

one_shot_result:
시장: 인공지능 반도체  
경쟁업체:  
- 엔비디아(NVIDIA): AI 연산에 최적화된 GPU를 중심으로 시장을 선도하며, 데이터센터 및 자율주행 등 다양한 AI 응용 분야에서 강력한 입지 확보  
- AMD: 고성능 GPU 및 CPU를 결합한 솔루션으로 AI 및 머신러닝 워크로드에 대응하며, 엔비디아와 경쟁 구도 형성  
- 구글(Google): 자체 개발한 TPU(Tensor Processing Unit)를 통해 클라우드 기반 AI 연산에 특화된 반도체 제공, AI 서비스 최적화에 집중  
- 인텔(Intel): AI 가속기 및 FPGA 기반 솔루션을 통해 데이터센터 및 엣지 컴퓨팅용 AI 반도체 시장 공략  
- 화웨이(Huawei): AI 칩셋인 Ascend 시리즈를 통해 중국 내수 및 글로벌 시장에서 AI 반도체 경쟁력 강화  

이들 기업은 삼성전자와 함께 AI 반도체 시장에서 기술력과 생태계 확장을 두고 경쟁하고 있습니다.


# **Few-shot** 프롬프팅

- **Few-shot 프롬프팅**은 AI 모델에게 2-5개의 예시를 제공하여 학습시키는 방법입니다

- 이 방식은 **Zero-shot**이나 **One-shot** 프롬프팅보다 더 우수한 성능을 보여주며, 복잡한 작업에서 특히 효과적입니다

- Few-shot 프롬프팅은 높은 성능을 제공하지만, 긴 프롬프트로 인한 **비용 증가**를 고려해야 합니다

In [11]:
# Example 데이터 준비 회사이름 나라이름 예시를 조금더 짧은 문장으로. 대신 여러개로 등록해서 사용하는 이유는 관리하기가 편하다. 프롬프트를 땡겨올수가 있으니까.
examples = """
시장: 스마트폰
경쟁업체: 
- 애플(미국): 프리미엄 시장 주도, iPhone으로 경쟁
- 샤오미(중국): 중저가 시장 강세, 글로벌 확장중
- 구글(미국): Pixel로 AI 기능 강조

시장: TV
경쟁업체:
- LG전자(한국): OLED 기술 경쟁
- Sony(일본): 프리미엄 시장 경쟁
- TCL(중국): 중저가 시장 공략
"""

# Few-shot 프롬프트를 Langfuse에 저장 (예시는 config에 저장) 결과가 표현자체가 더 간결해졌다. 컨텍스트까지 제공하면 더 자세하게 제공된다.
langfuse.create_prompt(
    name="competitor-analysis-few-shot",
    type="text",
    prompt="""다음은 여러 시장에서 삼성전자의 경쟁업체를 설명하는 예시들이다:

{{examples}}

이제 다음 시장에서 삼성전자의 경쟁업체를 설명해주세요:
시장: {{topic}}""",
    labels=["production"],
    tags=["competitor", "analysis", "few-shot"],
    config={
        "examples": examples
    }
)


In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Few-shot 프롬프트 템플릿 사용
few_shot_template = langfuse.get_prompt("competitor-analysis-few-shot")

# Langfuse 프롬프트를 LangChain과 통합
few_shot_prompt = PromptTemplate.from_template(
    few_shot_template.get_langchain_prompt()
)
few_shot_prompt.metadata={"langfuse_prompt": few_shot_template}  # Langfuse 자동 링크를 위한 메타데이터

# few_shot_prompt 적용한 체인 생성
chain = few_shot_prompt | llm | StrOutputParser()

# Few-shot 실행
topic = "인공지능 반도체"
few_shot_result = chain.invoke(
    input={
        "examples": few_shot_template.config.get("examples"),
        "topic": topic
    },
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백
)

print(f"few_shot_result:")
print(few_shot_result)

few_shot_result:
시장: 인공지능 반도체  
경쟁업체:  
- 엔비디아(미국): GPU 기반 AI 연산 시장 선도, 데이터센터 및 자율주행용 AI 칩 강세  
- AMD(미국): 고성능 GPU 및 CPU 통합 솔루션 제공, AI 및 머신러닝 가속화  
- 화웨이(중국): 자체 개발한 AI 칩(Ascend 시리즈)으로 중국 내수 및 글로벌 시장 공략  
- 텐센트(중국): AI 반도체 개발 투자 확대, 클라우드 AI 서비스와 연계  
- 인텔(미국): AI 전용 프로세서 및 FPGA 개발, 데이터센터 AI 솔루션 강화


`(3) FewShotChatMessagePromptTemplate 사용`

* FewShotChatMessagePromptTemplate는 LangChain에서 제공하는 템플릿으로, **미리 정의된 고정된 예제들(Fixed Examples)** 을 프롬프트에 포함시켜 모델이 일관된 형식과 품질의 응답을 생성할 수 있도록 돕습니다.

* 이 방식은 특히 특정 형식이나 구조를 가진 출력이 필요한 경우(예: JSON 형식, 특정 분석 리포트 형식 등) 매우 유용하며, 예제들이 고정되어 있어 결과의 일관성을 보장할 수 있습니다.

* 단, 고정된 예제를 사용하기 때문에 상황에 따라 유연하게 대응하기 어려울 수 있으며, 모든 케이스를 커버하기 위해서는 신중한 예제 선택이 필요합니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from textwrap import dedent # text의 모든 줄에서 같은 선행 공백을 제거하는 함수, 앞부분 공백만 사라짐

# 예시 데이터 정의 : 뉴스 텍스트(input) + 키워드 추출 결과 (output)
examples = [
    {
        "input": dedent("""
                        정부는 의과대학 입학 정원을 2000명 증가시킬 계획의 세부사항을 이달 20일에 공개할 예정이다. 
                        지역별 의료 서비스 향상과 소규모 의과대학의 발전을 목표로, 지역 중심의 국립대학 및 소형 의과대학의 
                        입학 정원이 최소한 두 배 가량 확대될 것으로 보인다.
                        """),
        "output": "의대 | 정원 | 확대"
    },
    {
        "input": dedent("""
                        세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 
                        전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.
                        """),
        "output": "세계보건기구 | 건강위기 | 국제"
    }
]

# 각 예시를 포맷팅할 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("assistant", "{output}")
])

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,      # 예시 포맷팅 템플릿
    examples=examples                   # 예시 데이터 리스트 -> 예시 포맷팅 템플릿에 적용
)

# Few-shot 프롬프트 출력 예시
print(few_shot_prompt.format())

Human: 
정부는 의과대학 입학 정원을 2000명 증가시킬 계획의 세부사항을 이달 20일에 공개할 예정이다. 
지역별 의료 서비스 향상과 소규모 의과대학의 발전을 목표로, 지역 중심의 국립대학 및 소형 의과대학의 
입학 정원이 최소한 두 배 가량 확대될 것으로 보인다.

AI: 의대 | 정원 | 확대
Human: 
세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 
전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.

AI: 세계보건기구 | 건강위기 | 국제


In [ ]:
# 최종 프롬프트 템플릿 생성 
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 텍스트에서 핵심 키워드 3개를 추출하는 전문가입니다."),
    few_shot_prompt,
    ("human", "{input}")
])
#앞에서 템플릿을 사용해놓으니까 다음것도 잘 진행된다.
# 프롬프트 템플릿 출력
print(final_prompt.format(input="테스트 뉴스 기사입니다."))

System: 당신은 뉴스 텍스트에서 핵심 키워드 3개를 추출하는 전문가입니다.
Human: 
정부는 의과대학 입학 정원을 2000명 증가시킬 계획의 세부사항을 이달 20일에 공개할 예정이다. 
지역별 의료 서비스 향상과 소규모 의과대학의 발전을 목표로, 지역 중심의 국립대학 및 소형 의과대학의 
입학 정원이 최소한 두 배 가량 확대될 것으로 보인다.

AI: 의대 | 정원 | 확대
Human: 
세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 
전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.

AI: 세계보건기구 | 건강위기 | 국제
Human: 테스트 뉴스 기사입니다.


In [ ]:
# 키워드 추출 체인 생성
chain = final_prompt | llm | StrOutputParser()

# 키워드 추출 체인 실행 실제로는 체인안에서 세개를 꺼내온다.
result = chain.invoke({
    "input": dedent("""삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 
                    이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 
                    세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.""")
})

print(result)

- Langfuse에서 구현 (메시지플레이스홀더 사용)

In [15]:
# Few-shot 프롬프트를 Langfuse에 생성 (Message Placeholder 사용)
langfuse.create_prompt(
    name="keyword-extractor-few-shot",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": """당신은 뉴스 텍스트에서 핵심 키워드를 추출하는 전문가입니다.
다음 지침을 따라주세요:
- 가장 중요한 키워드 3개를 추출
- 키워드는 ' | ' 로 구분
- 명사 위주로 추출
- 고유명사와 핵심 개념 우선"""
        },
        {
            "type": "placeholder",  #예시는 메세지플레이스홀더타입으로 사용
            "name": "few_shot_examples"  # Few-shot 예시들이 들어갈 placeholder 이거안써도 잘 파악하겟지만 예시를 쓴다.
        },
        {
            "role": "user",
            "content": "다음 뉴스 텍스트에서 핵심 키워드를 추출해주세요:\n\n{{input_text}}"
        }
    ],
    labels=["production"],
    tags=["keyword", "extraction", "few-shot", "news"],
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.1,  # 일관성을 위해 낮은 temperature
        "max_tokens": 100
    }
)

In [ ]:

# Langfuse 프롬프트 가져오기
prompt = langfuse.get_prompt("keyword-extractor-few-shot", type="chat")

# LangChain과 통합하여 실행
langchain_prompt = ChatPromptTemplate.from_messages(
    prompt.get_langchain_prompt()
)

# 메타데이터 설정 이거해야 모델이 걸린다
langchain_prompt.metadata = {"langfuse_prompt": prompt}

# 모델 초기화
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-mini"),
    temperature=prompt.config.get("temperature", 0.1),
    max_completion_tokens=prompt.config.get("max_tokens", 100)
)

# 체인 생성
chain = langchain_prompt | model

# Few-shot 프롬프트 실행

# 뉴스 텍스트 정의
news_text = dedent("""
정부는 내년부터 전기차 구매 보조금을 기존 대비 30% 확대하기로 결정했다.
탄소 중립 목표 달성과 친환경 자동차 산업 육성을 위한 조치로, 
개인 구매자는 최대 800만원, 법인은 최대 500만원까지 지원받을 수 있다.
""")


# Few-shot 예시 데이터 정의
few_shot_examples = [
    {
        "role": "user",
        "content": dedent("""
                        정부는 의과대학 입학 정원을 2000명 증가시킬 계획의 세부사항을 이달 20일에 공개할 예정이다. 
                        지역별 의료 서비스 향상과 소규모 의과대학의 발전을 목표로, 지역 중심의 국립대학 및 소형 의과대학의 
                        입학 정원이 최소한 두 배 가량 확대될 것으로 보인다.
                        """)
    },
    {
        "role": "assistant",
        "content": "의대 | 정원 | 확대"
    },
    {
        "role": "user", 
        "content": dedent("""
                        세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 
                        전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.
                        """)
    },
    {
        "role": "assistant",
        "content": "세계보건기구 | 건강위기 | 국제협력"
    },
    {
        "role": "user",
        "content": dedent("""
                        삼성전자가 새로운 갤럭시 스마트폰 시리즈를 내년 1분기에 출시할 예정이라고 발표했다.
                        인공지능 기능이 대폭 강화되고 배터리 수명도 20% 향상될 것으로 예상된다.
                        """)
    },
    {
        "role": "assistant", 
        "content": "삼성전자 | 갤럭시 | 인공지능"
    }
]
#아래 체인으로 실제 뉴스를 전달하면 아래로  Inputext로 연결이 되고 예시는 플레이스홀더로 전달된다 6개의 주고받은것들이 플레이스홀더로. 메타데이터는 통과되고 세개 키워드
#시스템메세지 다음에 플레이스홀더에 적히면서 모델이 이해를한다. 유저메세지는 유저가보내는것을 알아서 히스토리처럼 기억하고
#유저가 입력을 했을때, 
response = chain.invoke(
    input={
        "input_text": news_text,
        "few_shot_examples": few_shot_examples
    },
    config={
        "callbacks": [langfuse_handler],
        "metadata": {
            "user_id": "user_123",
            "task": "keyword_extraction",
            "method": "few_shot_learning"
        }
    }
)

print(response.content)

Placeholders ['few_shot_examples'] have not been resolved. Pass them as keyword arguments to compile().


전기차 | 보조금 | 탄소중립


`(4) Dynamic Few-Shot Prompting`

* **Dynamic Few-Shot Prompting**은 상황에 따라 적절한 예시를 동적으로 선택하여 사용하는 고급 프롬프팅 기법으로, **BaseExampleSelector**를 통해 입력값과 가장 연관성이 높은 예시들을 자동으로 선별합니다.

* 대표적으로 **SemanticSimilarityExampleSelector**는 의미적 유사도를 기반으로 예시를 선택하며, 이를 통해 주어진 입력 상황에 가장 적합한 예시들만을 효율적으로 활용할 수 있습니다.

* **example_prompt**를 통해 선택된 예시들을 AI 시스템이 이해하기 쉬운 형태(예: human-AI 대화 , human-function call)로 변환하여 더 효과적인 학습과 응답 생성이 가능하게 합니다.


- **장점**

    - 상황에 맞는 가장 연관성 높은 예시만을 선택적으로 활용할 수 있다
    - 프롬프트의 길이를 효율적으로 관리할 수 있다
    - 응답의 일관성과 품질을 향상시킬 수 있다

In [ ]:
from langchain_ollama import OllamaEmbeddings 
from langchain_core.vectorstores import InMemoryVectorStore # type: ignore

# 고객 문의 유형별 응대 예시 데이터 랭체인에 구현돼있다. 임베딩모델을 사용한다. 
# few shot 예시도 상황에 따른 질문에 따라서 사용자 질문에관련된 faq만 가져와서 프롬프트에 넣어서  rag처럼 컨텍스트 가져오는 아이디어를 가져오는 개념이다.
examples = [
    {
        "input": "환불 절차가 어떻게 되나요?",
        "output": "환불 절차는 다음과 같습니다:\n1. 구매내역에서 환불을 신청해주세요\n2. 반품 상품을 발송해주세요\n3. 상품 검수 후 3-5일 내 환불이 완료됩니다"
    },
    {
        "input": "배송이 늦어지고 있어요", 
        "output": "불편을 드려 죄송합니다. 주문번호를 알려주시면 배송 상태를 즉시 확인해드리겠습니다."
    },
    {
        "input": "옷 사이즈가 안 맞아요",
        "output": "사이즈 교환은 무료로 진행됩니다. 교환 신청 후 동일 상품의 다른 사이즈로 발송해드리겠습니다."
    },
    {
        "input": "제품이 불량이에요",
        "output": "불편을 드려 대단히 죄송합니다. 불량 부분 사진과 함께 1:1 문의에 접수해주시면 빠르게 처리해드리겠습니다."
    },
    {
        "input": "주문을 취소하고 싶어요",
        "output": "주문 취소는 배송 전까지 가능합니다. 마이페이지에서 주문 취소를 진행해주시거나 고객센터로 연락주세요."
    },
    {
        "input": "할인 쿠폰이 적용이 안 돼요",
        "output": "쿠폰 사용 조건을 확인해주세요. 최소 구매 금액이나 적용 상품에 제한이 있을 수 있습니다."
    },
    {
        "input": "회원가입이 안 돼요",
        "output": "회원가입 시 필수 정보를 모두 입력해주세요. 문제가 지속되면 고객센터로 연락주시기 바랍니다."
    }
]

# 예시 데이터를 벡터화할 텍스트로 변환  질문 답변을 둘다 하나의 임베딩으로 나눴다.근데 따로 질문 , 답변 임베딩 따로하는게 낫지 않나? 이런 생각.
to_vectorize = [" ".join(example.values()) for example in examples]

# Ollama 임베딩 모델 생성 텍스트를 올라마 사용해서 벡터임베딩저장할거다
embeddings = OllamaEmbeddings(model="bge-m3")

# 벡터 스토어 생성 #벡터스토어는 크로마나 파이스사용하면되고
vector_store = InMemoryVectorStore.from_texts(
    to_vectorize,    # 벡터화할 텍스트 리스트
    embeddings,      # 임베딩 모델
    metadatas=examples    # 메타데이터: 예시 데이터 , 벡터스토어에 7개의 예시를 벡터화해놓는다.
    )

# VectorStore에 저장된 Document 개수 확인 랭체인의 다큐멘트로 저장되는데 원본 데이터가 메타데이터가된다.
print(f"VectorStore에 저장된 Document 개수: {len(vector_store.store.keys())}")

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

In [ ]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

# 유사한 2개의 예시를 선택하는 selector 생성, 맥락적 의미를 가져오는 도구를 사용해서 2개를 벡터 비슷한거를 가져오는거다. 
#랭체인의 구현에서 그걸 가져온다. 예시를 벡터에서 가져온다. 벡터스토어에서 질문을 임베딩해서, 벡터스토어에서 검색해서 가져온다. 리턴할때 메타데이터를 리턴한다.
#랭체인에 메모리에 잠깐 저장해놓는 거를 가져온다.
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vector_store,
    k=2
)

# 선택된 예시 확인
selected_examples = example_selector.select_examples({"input":"상품이 파손되어 왔어요"})

from pprint import pprint
pprint(selected_examples)

In [ ]:
# 챗봇 프롬프트 템플릿 생성, 이걸 퓨샷 예제 가져오는 거를 똑같이하는데, 차이는 이그젬플을 셀렉터로 유사한거를 가져오는걸로한다.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("assistant", "{output}")
    ])
)

pprint(few_shot_prompt.invoke({"input": "상품이 파손되어 왔어요"}).to_messages())

In [ ]:
# 최종 프롬프트 생성 , 예제를 처리해주는 별도의 템플릿을 처리한다.
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절하고 전문적인 고객 서비스 담당자입니다."),
    few_shot_prompt,
    ("human", "{input}")
])

pprint(final_prompt.invoke({"input": "상품이 파손되어 왔어요"}).to_messages())

In [ ]:
# 챗봇 체인 생성 #이걸 체인으로 구성한다. 그러면 최종 답변이 나온다. 여기서는 시스템프롬프트를 더 명확하게해야한다. 내용을 제대로할건지 구조만 할건지. 불량일때랑 파손일때랑 다를수도있으니까
chain = final_prompt | llm | StrOutputParser()

# 체인 실행
response = chain.invoke({
    "input": "상품이 파손되어 왔어요"
})

pprint(response)

- langfuse에서 구현 (메시지플레이스홀더 사용)

In [ ]:
# Langfuse에 Semantic Few-shot 프롬프트 생성 랭퓨즈가 로그를 확인하는게 무료여서 랭스미스는 유료여서 
langfuse.create_prompt(
    name="customer-service-semantic-few-shot",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 친절하고 전문적인 고객 서비스 담당자입니다. 고객의 문의에 정확하고 도움이 되는 답변을 제공해주세요."
        },
        {
            "type": "placeholder",
            "name": "selected_examples"  # 유사도 기반으로 선택된 예시들
        },
        {
            "role": "user",
            
            "content": "{{customer_inquiry}}"  # 실제 고객 문의
        }
    ],
    labels=["production"],
    tags=["customer-service", "semantic-similarity", "few-shot"],
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7,
        "max_tokens": 300
    }
)

In [ ]:
# 선택된 예시 확인
selected_examples = example_selector.select_examples({"input":"상품이 파손되어 왔어요"})

from pprint import pprint
pprint(selected_examples)

In [ ]:
# 선택된 예시들을 메시지 형태로 변환
selected_examples_messages = []
for example in selected_examples:
    selected_examples_messages.extend([
        {"role": "user", "content": example["input"]},
        {"role": "assistant", "content": example["output"]}
    ])

for message in selected_examples_messages:
    print(message)
    print("-" * 80)

In [ ]:
# 동적 Few-shot 프롬프트 실행 함수
def run_semantic_few_shot(customer_inquiry, k=2):
    """고객 문의에 대해 유사도 기반 Few-shot 프롬프트 실행"""
    
    # 유사한 예시들을 선택
    selected_examples_raw = example_selector.select_examples({"input": customer_inquiry})
    
    # 선택된 예시들을 메시지 형태로 변환
    selected_examples_messages = []
    for example in selected_examples_raw:
        selected_examples_messages.extend([
            {"role": "user", "content": example["input"]},
            {"role": "assistant", "content": example["output"]}
        ])
    
    # Langfuse 프롬프트 가져오기
    prompt = langfuse.get_prompt("customer-service-semantic-few-shot", type="chat")

    langchain_prompt = ChatPromptTemplate.from_messages(
        prompt.get_langchain_prompt()
    )
    langchain_prompt.metadata = {"langfuse_prompt": prompt}


    compiled_prompt = prompt.compile(
        customer_inquiry=customer_inquiry,
        selected_examples=selected_examples_messages
    )


    # 모델 초기화
    model = ChatOpenAI(
        model=prompt.config.get("model", "gpt-4.1-mini"),
        temperature=prompt.config.get("temperature", 0.7),
        max_completion_tokens=prompt.config.get("max_tokens", 300)
    )

    # LangChain 체인 설정
    chain = langchain_prompt | model

    # 체인 실행
    response = chain.invoke(
        input={
            "customer_inquiry": customer_inquiry,
            "selected_examples": selected_examples_messages
        },
        config={"callbacks": [langfuse_handler]}
    )
    
    return response, selected_examples_messages

# 테스트 실행
test_inquiry = "상품이 파손되어 왔어요"
response, selected_examples = run_semantic_few_shot(test_inquiry)

In [ ]:
print(response.content)
print("=" * 80)
for message in selected_examples:
    print(message)
    print("-" * 80)